# Get Up and Running Quickly

## 🌍 Overview

This quickstart aims to give you a small illustration of what ZenML can do. We will:

- Import some data from a public dataset (Adult Census Income), then train two models (SGD and Random Forest)
- Compare and evaluate which model performs better, and deploy the best one.
- Run a prediction on the deployed model.

Along the way we will also show you how to:

- Automatically version, track, and cache data, models, and other artifacts,
- Track model hyperparameters and metrics in an experiment tracking tool,

This will give you enough to get started building your own ZenML Pipelines.
Let's dive in!


DIAGRAM SHOWING THE FLOW


- introduction to what the quickstart is about
- what will be covered here / what we'll do

## Run on Colab

You can use Google Colab to see ZenML in action, no signup / installation
required!

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](
https://colab.research.google.com/github/zenml-io/zenml/blob/main/examples/quickstart/new_quickstart/new_quickstart.ipynb)

# 1. Install Requirements

Let's install ZenML to get started. First we'll install the latest version of
ZenML as well as the two integrations we'll need for this quickstart: `sklearn`
and `mlflow`.

In [ ]:
# TODO: add things relating to cloudflare pipelines etc and zenml installation
# !pip install -q -e "../../../.[dev]"

In [ ]:
# !zenml integration install sklearn mlflow -y
!zenml init
# %pip install pyngrok pyparsing==2.4.2  # required for Colab #TODO: STILL NEEDED?

# # automatically restart kernel
# import IPython
# IPython.Application.instance().kernel.do_shutdown(restart=True)

Please wait for the installation to complete before running subsequent cells. At the end of the installation, the notebook kernel will automatically restart.

# 2. Import Data

We'll start off by importing our data. In this quickstart we'll be working with
[the Adult Census Income](https://archive.ics.uci.edu/dataset/2/adult) dataset
which is publicly available on the UCI Machine Learning Repository. The task is
to predict whether a person makes over $50k a year based on a number of
features. These features are things like age, work class, education level,
marital status, occupation, relationship, race, sex, capital gain, capital loss,
hours per week, and native country.

When you're getting started with a machine learning problem you'll want to do
something similar to this: import your data and get it in the right shape for
your training. ZenML mostly gets out of your way when you're writing your Python
code, as you'll see from the following cell.

In [8]:
from typing import Annotated, Tuple

import pandas as pd
from sklearn.model_selection import train_test_split

from zenml import step


@step
def training_data_loader() -> (
    Tuple[
        Annotated[pd.DataFrame, "X_train"],
        Annotated[pd.DataFrame, "X_test"],
        Annotated[pd.Series, "y_train"],
        Annotated[pd.Series, "y_test"],
    ]
):
    """Load the Census Income dataset as tuple of Pandas DataFrame / Series."""
    # Load the dataset
    url = "https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data"
    column_names = [
        "age",
        "workclass",
        "fnlwgt",
        "education",
        "education-num",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "capital-gain",
        "capital-loss",
        "hours-per-week",
        "native-country",
        "income",
    ]
    data = pd.read_csv(
        url, names=column_names, na_values="?", skipinitialspace=True
    )

    # Drop rows with missing values
    data = data.dropna()

    # Encode categorical features and drop original columns
    categorical_cols = [
        "workclass",
        "education",
        "marital-status",
        "occupation",
        "relationship",
        "race",
        "sex",
        "native-country",
    ]
    data = pd.get_dummies(data, columns=categorical_cols, drop_first=True)

    # Encode target feature
    data["income"] = data["income"].apply(
        lambda x: 1 if x.strip() == ">50K" else 0
    )

    # Separate features and target
    X = data.drop("income", axis=1)
    y = data["income"]

    # Split the dataset into train and test sets
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=0.2, random_state=42
    )
    return X_train, X_test, y_train, y_test

We download the data, dropping some columns and then splitting it up into train
and test sets. The whole function is decorated with the `@step` decorator, which
tells ZenML to track this function as a step in the pipeline. This means that
ZenML will automatically version, track, and cache the data that is produced by
this function. This is a very powerful feature, as it means that you can
reproduce your data at any point in the future, even if the original data source
changes or disappears.

You'll also notice that we have included type hints for the outputs
to the function. These are not only useful for anyone reading your code, but
help ZenML process your data in a way appropriate to the specific data types.
We're using the `Annotated` type hint here, which is a special type hint that
allows us to give a name to our output. This is useful for when you have
multiple outputs from a single step and you might want to access a specific
output by key later on.

ZenML is built in a way that allows you to experiment with your data and build
your pipelines as you work, so if you want to call this function to see how it
works, you can just call it directly. Here we take a look at the first few rows
of your training dataset.

In [9]:
X_train, X_test, y_train, y_test = training_data_loader()
X_train.head()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,...,native-country_Portugal,native-country_Puerto-Rico,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia
19863,53,168539,5,0,0,70,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
24342,49,56841,13,0,0,70,False,False,False,True,...,False,False,False,False,False,False,False,True,False,False
10027,28,154571,10,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,False,False,False
25710,60,188236,6,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False
13824,53,87158,9,0,0,40,False,True,False,False,...,False,False,False,False,False,False,False,True,False,False


Everything looks as we'd expect and the values are all in the right format. We
can shift to training some models now! 🥳

# 3. Train Models

Now that we have our data it makes sense to train some models to get a sense of
how difficult the task is. The Census Income
dataset is sufficiently large and complex that it's unlikely we'll be able to
train a model that behaves perfectly since the problem is inherently complex,
but we can get a sense of what a reasonable baseline looks like.

We'll start with two simple models, a SGD Classifier and a Random Forest
Classifier, both batteries-included from `sklearn`. We'll train them both on the
same data and then compare their performance.

Since we're starting our work properly, it makes sense to start tracking the
experimentation that we're doing. ZenML integrates with MLflow to make this
easy. This happens out of the box when using our experiment tracker integration
and stack components. We'll see how this works below, but first let's set up
ZenML to know that it should use the MLFlow experiment tracker.

In [ ]:
# Register the MLflow experiment tracker
!zenml experiment-tracker register mlflow --flavor=mlflow

# Register a new stack with our experiment tracker
!zenml stack register quickstart -a default\
                                 -o default\
                                 -e mlflow

!zenml stack set quickstart

At this point our stack looks like this. We can now write the steps where we'll
train our models, making sure to specify the name of our experiment tracker in
the `@step` decorator. We could specify this manually using a string, but
instead we'll use the ZenML `Client` to access the name of our active stack's
experiment tracker.

In [3]:
import mlflow

from sklearn.base import ClassifierMixin
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier

from zenml.client import Client

experiment_tracker = Client().active_stack.experiment_tracker


@step(experiment_tracker=experiment_tracker.name)
def random_forest_trainer_mlflow(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> ClassifierMixin:
    """Train a sklearn Random Forest classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hyperparams and metrics to MLflow
    model = RandomForestClassifier()
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    train_acc = model.score(X_train.to_numpy(), y_train.to_numpy())
    print(f"Train accuracy: {train_acc}")
    return model


@step(experiment_tracker=experiment_tracker.name)
def sgd_trainer_mlflow(
    X_train: pd.DataFrame,
    y_train: pd.Series,
) -> ClassifierMixin:
    """Train a SGD classifier and log to MLflow."""
    mlflow.sklearn.autolog()  # log all model hyperparams and metrics to MLflow
    model = SGDClassifier()
    model.fit(X_train.to_numpy(), y_train.to_numpy())
    train_acc = model.score(X_train.to_numpy(), y_train.to_numpy())
    print(f"Train accuracy: {train_acc}")
    return model

Our two training steps both return different kinds of `sklearn` classifier
models, so we use the generic `ClassifierMixin` type hint for the return type.

The end goal of this quick baseline evaluation is to understand which of the two
models performs better. We'll use the `evaluator` step to compare the two
models. This step takes in the two models we trained above, and compares them on
the test data we created earlier. It returns whichever model performs best.

In [4]:
@step
def evaluator(
    X_test: pd.DataFrame,
    y_test: pd.Series,
    model1: ClassifierMixin,
    model2: ClassifierMixin,
) -> ClassifierMixin:
    """Calculate the accuracy on the test set and return the best model of two."""
    test_acc1 = model1.score(X_test.to_numpy(), y_test.to_numpy())
    test_acc2 = model2.score(X_test.to_numpy(), y_test.to_numpy())
    print(f"Test accuracy ({model1.__class__.__name__}): {test_acc1}")
    print(f"Test accuracy ({model2.__class__.__name__}): {test_acc2}")
    return model1 if test_acc1 > test_acc2 else model2

We'll likely want to use our model in the future so instead of simply outputting
the model we'll use the MLflow model registry to store it. This allows us to
version the model for retrieval and use later on as well as to use other
functionality made possible within the MLflow dashboard. This step is a bit
different from the ones listed above in that we're using a pre-built ZenML step
instead of just writing our own. You'll often come across these pre-built steps
for common workflows.

In [5]:
from zenml.integrations.mlflow.steps.mlflow_registry import (
    mlflow_register_model_step,
)

model_name = "zenml-quickstart-model"

register_model = mlflow_register_model_step.with_options(
    parameters=dict(
        name=model_name,
        description="The first run of the Quickstart pipeline.",
    )
)

We're now at the point where can bring all these steps together into a single
pipeline, the top-level organising entity for code in ZenML. Creating such a pipeline is
as simple as adding a `@pipeline` decorator to a function. This specific
pipeline doesn't return a value, but that option is available to you if you need.

In [10]:
from zenml import pipeline


@pipeline
def train_and_register_model_pipeline() -> None:
    """Train a model."""
    register_model.after(evaluator)

    X_train, X_test, y_train, y_test = training_data_loader()
    model1 = random_forest_trainer_mlflow(X_train=X_train, y_train=y_train)
    model2 = sgd_trainer_mlflow(X_train=X_train, y_train=y_train)
    best_model = evaluator(
        X_test=X_test, y_test=y_test, model1=model1, model2=model2
    )
    register_model(best_model)

Notice how we specify that we want the model registration to run *after* the
evaluation step. This is because we won't have a model ready for registration
until the evaluation has taken place. ZenML automatically tries to run steps in
parallel, so sometimes if you have this kind of sequencing you need to do then
you'll need to specify it explicitly.

We've used the built-in MLflow registry to store our model, but ZenML doesn't
yet know that we want to use the MLflow flavor of the model registry stack
component in our stack. Let's add that now and update our stack.

In [ ]:
# Register the MLflow model registry
!zenml model-registry register mlflow --flavor=mlflow

# Update our stack to include the model registry
!zenml stack update quickstart -r mlflow

DIAGRAM SHOWING THE NEW STACK

We're ready to run the pipeline now, which we can do just -- as with the step -- by calling the
pipeline function itself.

In [11]:
train_and_register_model_pipeline()

Registered pipeline train_and_register_model_pipeline (version 2).
Running pipeline train_and_register_model_pipeline on stack quickstart (caching enabled)
Step training_data_loader has started.
Step training_data_loader has finished in 9.887s.
Step random_forest_trainer_mlflow has started.


2023/07/05 10:30:18 INFO mlflow.tracking.fluent: Experiment with name 'train_and_register_model_pipeline' does not exist. Creating a new experiment.
2023/07/05 10:30:19 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/07/05 10:31:02 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/Users/strickvl/.pyenv/versions/3.10.11/envs/quickstart/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


Train accuracy: 0.9999171121886526
Step random_forest_trainer_mlflow has finished in 47.507s.
Step sgd_trainer_mlflow has started.


2023/07/05 10:31:07 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.


Train accuracy: 0.7788967632309669
Step sgd_trainer_mlflow has finished in 21.729s.
Step evaluator has started.
Test accuracy (RandomForestClassifier): 0.8508204873197415
Test accuracy (SGDClassifier): 0.7821979114868225
Step evaluator has finished in 1.650s.
Step mlflow_register_model_step has started.
No registered model with name zenml-quickstart-model found. Creating a newregistered model.
MLflow model registry does not take a version as an argument. Registering a new version for the model 'zenml-quickstart-model' a version will be assigned automatically.


2023/07/05 10:31:33 INFO mlflow.tracking._model_registry.client: Waiting up to 300 seconds for model version to finish creation.                     Model name: zenml-quickstart-model, version 1


Registered model zenml-quickstart-model with version 1 from source file:///Users/strickvl/Library/Application Support/zenml/local_stores/d32e1983-2602-4334-968a-6ee66c2e1783/mlruns/289096806505521610/a816b9b0d8ed47eea4b78634fc246393/artifacts/model.
Step mlflow_register_model_step has finished in 1.015s.
Pipeline run train_and_register_model_pipeline-2023_07_05-08_30_07_764991 has finished in 1m26s.
Pipeline visualization can be seen in the ZenML Dashboard. Run zenml up to see your pipeline!


You can see from the logs already how our model training went: the
`RandomForestClassifier` performed considerably better than the `SGDClassifier`,
so that will have been the model that was returned from the evaluation step and
then registered with the MLflow model registry.

MAYBE SHOW THE ZENML DASHBOARD HERE

ALSO MAYBE SHOW THE MLFLOW UI + HOW TO ACCESS IT HERE

In [ ]:
# Set up Cloudflare tunnel
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb

!zenml up --port 8237 & cloudflared tunnel --url http://localhost:8237

[[[And you can use `wget -qO- http://localhost:55555/quicktunnel` to get the
magic hostname.]]]

TODO: Reimplement this natively under the hood

We're using MLflow for our experiment tracking. If you'd like to inspect the
MLflow dashboard to see your experiments and what's been logged so far, run the
following cell. This cell will spin up a local server that you can access via
the link mentioned after the "Listening at:" `INFO` log statement.

In [ ]:
# TODO: Add cloudflare tunnel for MLflow UI

In [12]:
import os
from zenml.integrations.mlflow.mlflow_utils import get_tracking_uri

os.environ["MLFLOW_TRACKING_URI"] = get_tracking_uri()
!mlflow ui --backend-store-uri $MLFLOW_TRACKING_URI

[2023-07-05 10:33:20 +0200] [11690] [INFO] Starting gunicorn 20.1.0
[2023-07-05 10:33:20 +0200] [11690] [INFO] Listening at: http://127.0.0.1:5000 (11690)
[2023-07-05 10:33:20 +0200] [11690] [INFO] Using worker: sync
[2023-07-05 10:33:20 +0200] [11691] [INFO] Booting worker with pid: 11691
[2023-07-05 10:33:20 +0200] [11692] [INFO] Booting worker with pid: 11692
[2023-07-05 10:33:20 +0200] [11693] [INFO] Booting worker with pid: 11693
[2023-07-05 10:33:20 +0200] [11694] [INFO] Booting worker with pid: 11694
^C
[2023-07-05 10:33:37 +0200] [11690] [INFO] Handling signal: int
[2023-07-05 10:33:38 +0200] [11693] [INFO] Worker exiting (pid: 11693)
[2023-07-05 10:33:38 +0200] [11692] [INFO] Worker exiting (pid: 11692)
[2023-07-05 10:33:38 +0200] [11691] [INFO] Worker exiting (pid: 11691)
[2023-07-05 10:33:38 +0200] [11694] [INFO] Worker exiting (pid: 11694)


Our pipeline above registered the best model with the MLflow model registry.
Whenever you register a model it also versions the model since it's likely that
you'll be iterating and improving your model over time.

We'll now turn to actually deploying our model and serving some predictions, for
which we'll need to specify the model version we want to use. You can specify
the version number manually but below we'll use the ZenML `Client` to get the
latest version number.

In [15]:
from zenml.client import Client

most_recentmodel_version_number = int(
    Client()
    .active_stack.model_registry.list_model_versions(metadata={})[0]
    .version
)
most_recentmodel_version_number

1

Now we've trained our model, and we've found the best one, we want to deploy it
and run some inference on the deployed model. We'll use the local MLflow model
deployer which once again comes with some pre-built ZenML steps to save you reinventing the wheel.

In [16]:
from zenml.integrations.mlflow.steps.mlflow_deployer import (
    mlflow_model_registry_deployer_step,
)
from zenml.integrations.mlflow.steps.mlflow_registry import (
    mlflow_register_model_step,
)
from zenml.model_registries.base_model_registry import (
    ModelRegistryModelMetadata,
)

model_deployer = mlflow_model_registry_deployer_step.with_options(
    parameters=dict(
        registry_model_name=model_name,
        registry_model_version=most_recentmodel_version_number,
    )
)

When you deploy a model this is usually something you want to remain available
and running for a long time, so ZenML automatically creates a background service
for your deployed model. We load the service (already created by the
`model_deployer` step) and then use it to make some predictions.

In [24]:
from zenml.services import BaseService
from zenml.client import Client


@step(enable_cache=False)
def prediction_service_loader() -> BaseService:
    """Load the model service of our train_and_register_model_pipeline."""
    client = Client()
    model_deployer = client.active_stack.model_deployer
    services = model_deployer.find_model_server(
        pipeline_name="train_and_register_model_pipeline",
        running=True,
    )
    return services[0]


@step
def predictor(
    service: BaseService,
    data: pd.DataFrame,
) -> Annotated[list, "predictions"]:
    """Run a inference request against a prediction service."""
    service.start(timeout=10)  # should be a NOP if already started
    print(f"Running predictions on data (single individual): {data.to_numpy()[0]}")
    prediction = service.predict(data.to_numpy())
    print(f"Prediction (for single example slice) is: {bool(prediction.tolist()[0])}")
    return prediction.tolist()

In [18]:
# Register the MLflow model deployer
!zenml model-deployer register mlflow --flavor=mlflow

# Register a new stack with the new stack components
!zenml stack update quickstart -d mlflow

Using the default local database.
Running with active workspace: 'default' (repository)
Running with active stack: 'quickstart' (repository)
⠋ Registering model deployer 'mlflow'...
⠙ Registering model deployer 'mlflow'...
⠹ Registering model deployer 'mlflow'...
Successfully registered model_deployer `mlflow`.
⠹ Registering model deployer 'mlflow'...
⠹ Registering model deployer 'mlflow'...

Using the default local database.
Running with active workspace: 'default' (repository)
⠋ Updating stack...
⠙ Updating stack...
⠹ Updating stack...
Stack `quickstart` successfully updated!
⠸ Updating stack...
⠸ Updating stack...



Once again there are some dependencies in terms of how the step needs to run, so
we specify those upfront. For example, the prediction service needs to be loaded
before we try to make predictions with it.

In [26]:
@pipeline
def deploy_and_predict() -> None:
    """Deploy the best model and run some predictions."""
    prediction_service_loader.after(model_deployer)
    predictor.after(prediction_service_loader)
    model_deployer()
    _, inference_data, _, _ = training_data_loader()
    model_deployment_service = prediction_service_loader()
    predictor(service=model_deployment_service, data=inference_data)

In [27]:
deploy_and_predict()

Registered pipeline deploy_and_predict (version 1).
Running pipeline deploy_and_predict on stack quickstart (caching enabled)
Step mlflow_model_registry_deployer_step has started.


Updating an existing MLflow deployment service: MLFlowDeploymentService[599fd11b-d4c9-46c6-8119-764b4f5ea707] (type: model-serving, flavor: mlflow)


Output()

MLflow deployment service started and reachable at:
    http://127.0.0.1:8002/invocations

Step mlflow_model_registry_deployer_step has finished in 46.857s.
Step training_data_loader has started.
Using cached version of training_data_loader.
Step prediction_service_loader has started.
Step prediction_service_loader has finished in 0.383s.
Step predictor has started.


Running predictions on data (example slice): [28 76714 15 0 0 55 False True False False False False False False False
 False False False False False False False False False False True False
 False False False True False False False False False False False False
 False False True False False False False True False False False False
 False False False True True False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False True False False]
Prediction (example slice) is: False
Step predictor has finished in 0.976s.
Pipeline run deploy_and_predict-2023_07_05-08_50_06_948298 has finished in 51.279s.
Pipeline visualization can be seen in the ZenML Dashboard. Run zenml up to see your pipeline!


If you recall, the purpose of this model was to predict whether or not someone
earns more than $50k per year. You can see a single example in the output above.
Given the features of a particular individual, the model predicts that they do
not earn more than $50k per year.

If we were interested in learning more about the model's predictions, we could
separately load the predictor service and use it to pass in some other data or
try things out. To load the predictor we can run:

In [30]:
predictor_service = deploy_and_predict.model.last_successful_run.steps[
    "prediction_service_loader"
].output.load()

predictor_service

MLFlowDeploymentService[599fd11b-d4c9-46c6-8119-764b4f5ea707] (type: model-serving, flavor: mlflow)

At this point, passing in some data is as simple as calling the `predict` method
on the predictor service. We can try this here:

In [41]:
print(
    f"Model predictions: {predictor_service.predict(X_test.to_numpy()[25:35])}"
)
print(f"Ground truth:      {y_test.to_numpy()[25:35]}")

Model predictions: [0 0 0 0 0 0 0 1 0 0]
Ground truth:      [1 1 1 0 0 1 0 1 0 0]


We're passing in some of our test data into the model and getting back the
predictions. You can already start to see some of the places where our
predictions are not matching the ground truth labels. This is to be expected but
we could potentially use this to now iterate on our models by adding more steps.

To get an overview of the models and model versions that we have registered and
deployed so
far, we can use the CLI to list these out.

In [42]:
!zenml model-registry models list

Using the default local database.
Running with active workspace: 'default' (repository)
Running with active stack: 'quickstart' (repository)
┏━━━━━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━┯━━━━━━━━━━┓
┃          NAME          │ DESCRIPTION │ METADATA ┃
┠────────────────────────┼─────────────┼──────────┨
┃ zenml-quickstart-model │             │          ┃
┗━━━━━━━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━┷━━━━━━━━━━┛


In [43]:
!zenml model-registry models list-versions zenml-quickstart-model

Using the default local database.
Running with active workspace: 'default' (repository)
Running with active stack: 'quickstart' (repository)
┏━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━━━┓
┃                    │               │ VERSION_DESCRIPTIO │                    ┃
┃        NAME        │ MODEL_VERSION │ N                  │ METADATA           ┃
┠────────────────────┼───────────────┼────────────────────┼────────────────────┨
┃ zenml-quickstart-m │ 1             │ The first run of   │ {'zenml_version':  ┃
┃        odel        │               │ the Quickstart     │ '0.41.0',          ┃
┃                    │               │ pipeline.          │ 'zenml_run_name':  ┃
┃                    │               │                    │ 'train_and_registe ┃
┃                    │               │                    │ r_model_pipeline-2 ┃
┃                    │               │                    │ 023_07_05-08_30_07 ┃
┃                    │               │           

In [44]:
!zenml model-deployer models list

Using the default local database.
Running with active workspace: 'default' (repository)
Running with active stack: 'quickstart' (repository)
┏━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━━━━━━━┯━━━━━━━━━━━━┓
┃        │                  │                  │ PIPELINE_STEP_NA │            ┃
┃ STATUS │ UUID             │ PIPELINE_NAME    │ ME               │ MODEL_NAME ┃
┠────────┼──────────────────┼──────────────────┼──────────────────┼────────────┨
┃   ✅   │ 599fd11b-d4c9-46 │ train_and_regist │                  │ model      ┃
┃        │ c6-8119-764b4f5e │ er_model_pipelin │                  │            ┃
┃        │ a707             │ e                │                  │            ┃
┗━━━━━━━━┷━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━━━━━━━┷━━━━━━━━━━━━┛


To view all this on the ZenML Dashboard, simply spin up the server again and
view the steps via the DAG visualiser and also browse the artifacts.

In [ ]:
# Set up Cloudflare tunnel
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb && dpkg -i cloudflared-linux-amd64.deb

!zenml up --port 8237 & cloudflared tunnel --url http://localhost:8237

## Congratulations!

You just built two ML pipelines! You trained two models, evaluated them against
a test set, registered the best one with the MLflow model registry, deployed it
and served some predictions. You also learned how to iterate on your models and
data by using some of the ZenML utility abstractions. You saw how to view your
artifacts and stacks via the CLI as well as the ZenML Dashboard.

And that is just the tip of the iceberg of what ZenML can do; check out the [**Integrations**](https://zenml.io/integrations) page for a list of all the cool MLOps tools that ZenML supports!

To improve upon the ML workflows we built in this quickstart, you could, for instance:
- [Deploy ZenML on the Cloud](https://docs.zenml.io/platform-guide/set-up-your-mlops-platform/deploy-zenml) to collaborate with your teammates,
- Experiment with more sophisticated models, such as [XGBoost](https://zenml.io/integrations/xgboost),
- Monitor your model's ongoing performance using [Evidently](https://zenml.io/integrations/evidently),
- Set up automated [Slack alerts](https://zenml.io/integrations/zen-ml-slack-integration) to get notified when data drift happens,
- Run the pipelines on scalable, distributed stacks like [Kubeflow](https://zenml.io/integrations/kubeflow).

## Where to go next

* If you have questions or feedback... 
  * Join our [**Slack Community**](https://zenml.io/slack-invite) and become part of the ZenML family!
* If this quickstart was a bit too quick for you...
  * Check out the [**Starter
    Guide**](https://docs.zenml.io/user-guide/starter-guide) which guides you
    through ZenML as well as some practical ways to get going on your MLOps journey.
* If you want to learn more about using or extending ZenML...
  * Check out our [**Docs**](https://docs.zenml.io/) or read through our code on [**Github**](https://github.com/zenml-io/zenml).
* If you want to quickly learn how to use a specific tool with ZenML...
  * Check out our collection of [**Examples**](https://github.com/zenml-io/zenml/tree/doc/hamza-misc-updates/examples).
* If you want to see some advanced ZenML use cases... 
  * Check out [**ZenML Projects**](https://github.com/zenml-io/zenml-projects), our collection of production-grade ML use-cases.